In [2]:
import praw
from collections import Counter
import datetime as dt
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time
import requests
from requests.exceptions import HTTPError

In [3]:
reddit = praw.Reddit(client_id='0VG-ZrOtnU3sHVmcPLfkxw',
                     client_secret='U_yiSSltVNcpi5RA48R12I9GqUZPPg',
                     user_agent='web:com.jupyter.redditanalysisforthesis:v1.0 (by /u/Educational_Play8158)')
                     
reddit.read_only = True

In [4]:
russell_comapnies = pd.read_csv('russell1000.csv')


remove_list = [' Corporation', 'Industries',' Inc.', ' Ltd.', ' Co.', '.', ',','&', '-','(Class A)','(Common)','(Series A)','  ','(Class B)','(Series B)','(Class C)','(Series C)', '(Class D)','(Series D)',' LLC', ' L.P.', ' Group', ' Holdings','Family Solutions']


def clean_company_name(name):
    for word in remove_list:
        name = name.replace(word, '')
    return name.strip()


russell_comapnies['Company'] = russell_comapnies['Company'].apply(clean_company_name)
tickers_list = russell_comapnies['Ticker']

In [5]:
tickers_list = russell_comapnies['Ticker']
tickers_list

0        MMM
1        TXG
2        AOS
3        ABT
4       ABBV
        ... 
1001    ZION
1002     ZTS
1003      ZM
1004      ZI
1005      ZS
Name: Ticker, Length: 1006, dtype: object

In [6]:
subreddits = ['wallstreetbets','investing','stocks', 'Robinhood','SecurityAnalysis']

In [7]:
# method 1 
from threading import Semaphore
request_semaphore = Semaphore(59)
results_cache = {}

def scrape_subreddit_company(difference, keyword, reddit):
    global results_cache
    
    cache_key = f"{keyword}"
    if cache_key in results_cache:
        print(f"Retrieved '{keyword}' from cache.")
        return results_cache[cache_key]
    
    print(f"Scraping for keyword: {keyword}")
    start_time = time.time()

    upper_limit = datetime.strptime('12/31/2023', '%m/%d/%Y')
    stop_limit = datetime.strptime('01/01/2015', '%m/%d/%Y')

    lower_limit = upper_limit - relativedelta(days=difference)
    data_list = []
    backoff_time = 1  

    while lower_limit >= stop_limit:
        with request_semaphore:
            try:
                search_query = f'{keyword} timestamp:{int(lower_limit.timestamp())}..{int(upper_limit.timestamp())}'
                response = reddit.subreddit(subreddit).search(search_query, time_filter='all', limit=100)
                
                for post in response:
                    data_list.append([
                        datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                        keyword,
                        post.title,
                        post.subreddit.display_name,
                        post.selftext
                    ])

                upper_limit = lower_limit
                lower_limit -= relativedelta(days=difference)
                backoff_time = 1 

            except praw.exceptions.RedditAPIException as e:
                print(f"Encountered an API exception: {e}")
                time.sleep(backoff_time)
                backoff_time *= 2

                time.sleep(1) 

    df = pd.DataFrame(data_list, columns=['Date', 'Keyword', 'Title', 'Subreddit', 'Content'])
    results_cache[cache_key] = df
    print(f"Scraping completed for keyword '{keyword}' in {time.time() - start_time} seconds")
    return df

In [21]:
from concurrent.futures import ThreadPoolExecutor
results_futures = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for subreddit in subreddits:
        for keyword in tickers_list:
            future = executor.submit(scrape_subreddit_company, 20, keyword, reddit)
            results_futures.append(future)

Scraping for keyword: PCTY
Scraping for keyword: PYPL
Scraping for keyword: PEGA
Scraping for keyword: PTON
Scraping for keyword: PENN
Scraping completed for keyword 'PYPL' in 1.2722220420837402 seconds
Scraping for keyword: PAG
Scraping completed for keyword 'PENN' in 1.2780890464782715 seconds
Scraping for keyword: PNR
Scraping completed for keyword 'PTON' in 1.2879180908203125 seconds
Scraping for keyword: PEN
Scraping completed for keyword 'PEGA' in 1.4761579036712646 seconds
Scraping for keyword: PEP
Scraping completed for keyword 'PCTY' in 1.6499018669128418 seconds
Scraping for keyword: PFGC
Scraping completed for keyword 'PEN' in 0.7446370124816895 seconds
Scraping for keyword: PRGO
Scraping completed for keyword 'PAG' in 0.8338689804077148 seconds
Scraping for keyword: WOOF
Scraping completed for keyword 'PEP' in 0.6392090320587158 seconds
Scraping for keyword: PFE
Scraping completed for keyword 'PNR' in 1.2465119361877441 seconds
Scraping for keyword: PCG
Scraping completed f

In [17]:
# method 2
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from threading import Lock
from concurrent.futures import ThreadPoolExecutor
import praw  # Assuming PRAW is used for Reddit API

cache_lock = Lock()
results_cache = {}

rate_limit_lock = Lock()
last_request_time = None
request_interval = 1  # seconds between requests to respect rate limits

def scrape_subreddit_company(difference, keyword, subreddit_name):
    global last_request_time

    with cache_lock:
        cache_key = f"{keyword}"
        if cache_key in results_cache:
            print(f"Retrieved '{keyword}' from cache.")
            return results_cache[cache_key]

    print(f"Scraping for keyword: {keyword}")
    start_time = time.time()

    upper_limit = datetime.strptime('04/08/2024', '%m/%d/%Y')
    stop_limit = datetime.strptime('01/01/2024', '%m/%d/%Y')
    lower_limit = upper_limit - relativedelta(days=difference)
    data_list = []

    while lower_limit >= stop_limit:
        with rate_limit_lock:
            current_time = time.time()
            if last_request_time is not None and (current_time - last_request_time) < request_interval:
                time.sleep(request_interval - (current_time - last_request_time))
            last_request_time = time.time()

        search_query = f'{keyword} timestamp:{int(lower_limit.timestamp())}..{int(upper_limit.timestamp())}'
        response = reddit.subreddit(subreddit_name).search(search_query, time_filter='all', limit=100)
        
        for post in response:
            data_list.append([
                datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d'),
                keyword,
                post.title,
                post.subreddit.display_name,
                post.selftext
            ])

        upper_limit = lower_limit
        lower_limit -= relativedelta(days=difference)

    df = pd.DataFrame(data_list, columns=['Date', 'Keyword', 'Title', 'Subreddit', 'Content'])
    
    with cache_lock:
        results_cache[cache_key] = df

    print(f"Scraping completed for keyword '{keyword}' in {time.time() - start_time} seconds")
    return df

# Example usage:
results_futures = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for subreddit in subreddits:
        for keyword in tickers_list:
            future = executor.submit(scrape_subreddit_company, 20, keyword, subreddit)
            results_futures.append(future)

for future in results_futures:
    try:
        result = future.result(timeout=30)
        if not result.empty:
            print(f"Result not empty for {result['Keyword'].iloc[0]}")
        else:
            print("Result is empty")
    except Exception as e:
        print(f"Error processing future: {e}")


Scraping for keyword: MMM
Scraping for keyword: TXG
Scraping for keyword: AOS
Scraping for keyword: ABT
Scraping for keyword: ABBV
Scraping completed for keyword 'AES' in 40.05814290046692 seconds
Scraping for keyword: AGL
Scraping completed for keyword 'AFRM' in 40.13859510421753 seconds
Scraping for keyword: AGNC
Scraping completed for keyword 'AFL' in 40.14314389228821 seconds
Scraping for keyword: ADC
Scraping completed for keyword 'AGCO' in 40.154037952423096 seconds
Scraping for keyword: AIG
Scraping completed for keyword 'A' in 39.89781403541565 seconds
Scraping for keyword: AL
Scraping completed for keyword 'MMM' in 30.20823383331299 seconds
Scraping for keyword: ACHC
Scraping completed for keyword 'AGNC' in 41.56999921798706 seconds
Scraping for keyword: APD
Scraping completed for keyword 'TXG' in 34.23177480697632 seconds
Scraping for keyword: ACN
Scraping completed for keyword 'ADC' in 44.793805837631226 seconds
Scraping for keyword: ABNB
Scraping completed for keyword 'AOS'

In [16]:
results_futures = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for subreddit in subreddits:
        for keyword in tickers_list:
            future = executor.submit(scrape_subreddit_company, 20, keyword, subreddit)
            results_futures.append(future)


for future in results_futures:
    try:
        result = future.result(timeout=30) 
        if not result.empty:
            print(f"Result not empty for {result['Keyword'].iloc[0]}")
        else:
            print("Result is empty")
    except Exception as e:
        print(f"Error processing future: {e}")

Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Scraping completed for keyword 'ADP' in 30.001224994659424 seconds
Scraping for keyword: AES
Scraping completed for keyword 'ADT' in 30.229411840438843 seconds
Scraping for keyword: AFRM
Scraping completed for keyword 'AAP' in 30.315107107162476 seconds
Scraping for keyword: AFL
Scraping completed for keyword 'WMS' in 35.368298053741455 seconds
Scraping for keyword: AGCO
Scraping completed for keyword 'ACM' in 35.14747595787048 seconds
Scraping for keyword: A
Scraping completed for keyword 'WMS' in 36.0490288734436 seconds
Scraping for keyword: AES
Scraping completed for keyword 'ADT' in 36.932106018066406 seconds
Scraping for keyword: AFRM
Scraping completed for keyword 'AAP' in 37.93463897705078 seconds
Scraping for keyword: AFL
Scraping completed for keyword 'ADP' in 38.98288702964783 seconds
Scraping for keyword: AGCO
Result

KeyboardInterrupt: 

In [19]:
dataframes = []  # List to store all the DataFrames

for future in results_futures:
    try:
        result = future.result(timeout=30)  # Wait for a maximum of 30 seconds for each future to return
        if not result.empty:
            print(f"Result not empty for {result['Keyword'].iloc[0]}")
            dataframes.append(result)  
        else:
            print("Result is empty")
    except Exception as e:
        print(f"Error processing future: {e}")

Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result is empty
Result i

NameError: name 'df' is not defined

,Date,Keyword,Title,Subreddit,Content
